In [53]:
import numpy as np
from load_data import load_data
from preprocessing import cv_preprocessing
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score


In [54]:
def sample_points(k):
    x = np.random.rand(k,50)
    y = np.random.choice([0, 1], size=k, p=[.5, .5]).reshape([-1,1])
    return x,y

In [55]:
x, y = sample_points(10)
print(x[0])
print(y[0])

[0.86394373 0.21040667 0.42008095 0.19229181 0.09156148 0.2132207
 0.64677299 0.1053125  0.91124258 0.73780975 0.70169806 0.92995671
 0.2617872  0.43504336 0.31040293 0.24137997 0.13662096 0.38940719
 0.01505598 0.76492295 0.28868551 0.21245143 0.41618471 0.05177993
 0.90976738 0.88638011 0.49866326 0.81841058 0.40843972 0.01431263
 0.44140437 0.39538452 0.26279042 0.84485992 0.89227722 0.18578891
 0.96283994 0.85916981 0.54286353 0.45695291 0.6396088  0.57687382
 0.4397507  0.03965205 0.09674    0.88135634 0.74251802 0.06435616
 0.87567421 0.5631718 ]
[1]


In [56]:
class MetaSGD(object):
    def __init__(self):
        
        #initialize number of tasks i.e number of tasks we need in each batch of tasks
        self.num_tasks = 2
        
        #number of samples i.e number of shots  -number of data points (k) we need to have in each task
        self.num_samples = 10

        #number of epochs i.e training iterations
        self.epochs = 10000
        
        #hyperparameter for the inner loop (inner gradient update)
        self.alpha = 0.0001
        
        #hyperparameter for the outer loop (outer gradient update) i.e meta optimization
        self.beta = 0.0001
       
        #randomly initialize our model parameter theta
        self.theta = np.random.normal(size=50).reshape(50, 1)
         
        #randomly initialize alpha with same shape as theta
        self.alpha = np.random.normal(size=50).reshape(50, 1)
      
    #define our sigmoid activation function  
    def sigmoid(self,a):
        return 1.0 / (1 + np.exp(-a))
    
    
    #now let us get to the interesting part i.e training :P
    def train(self):
        
        #for the number of epochs,
        for e in range(self.epochs):        
            
            self.theta_ = []
            
            #for task i in batch of tasks
            for i in range(self.num_tasks):
               
                #sample k data points and prepare our train set
                XTrain, YTrain = sample_points(self.num_samples)
                print(f"XTrain={XTrain.shape}")
                print(f"YTrain={YTrain.shape}")
                a = np.matmul(XTrain, self.theta)
                print(f"a={a.shape}")
                YHat = self.sigmoid(a)
                print(f"YHat={YHat.shape}")
                #since we are performing classification, we use cross entropy loss as our loss function
                loss = ((np.matmul(-YTrain.T, np.log(YHat)) - np.matmul((1 -YTrain.T), np.log(1 - YHat)))/self.num_samples)[0][0]
                print(f"loss={loss.shape}")

                #minimize the loss by calculating gradients
                gradient = np.matmul(XTrain.T, (YHat - YTrain)) / self.num_samples
                print(f"gradient={gradient.shape}")

                #update the gradients and find the optimal parameter theta' for each of tasks
                self.theta_.append(self.theta - (np.multiply(self.alpha,gradient)))
                
     
            #initialize meta gradients
            meta_gradient = np.zeros(self.theta.shape)
                        
            for i in range(self.num_tasks):
            
                #sample k data points and prepare our test set for meta training
                XTest, YTest = sample_points(10)

                print("XTest", XTest.shape)
                print("self.theta_[i]", self.theta_[i].shape)
                #predict the value of y
                a = np.matmul(XTest, self.theta_[i])
                
                YPred = self.sigmoid(a)
                           
                #compute meta gradients
                meta_gradient += np.matmul(XTest.T, (YPred - YTest)) / self.num_samples

            #update our randomly initialized model parameter theta with the meta gradients
            self.theta = self.theta-self.beta*meta_gradient/self.num_tasks
                       
            #update our randomly initialized hyperparameter alpha with the meta gradients
            self.alpha = self.alpha-self.beta*meta_gradient/self.num_tasks
               

In [57]:

df_preprocessed, features, target_feature = load_data()
df_preprocessed = df_preprocessed.dropna(subset=['target_binary_intrusion'], how='any')

X_train, X_test, y_train, y_test = train_test_split(df_preprocessed[features], df_preprocessed['target_binary_intrusion'],
                                      test_size=0.15, stratify=df_preprocessed['target_binary_intrusion'])
X_train, X_test = cv_preprocessing(X_train, X_test)

model = MetaSGD()
#model.train(X_train, y_train, X_test, y_test)
model.train()

XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gr

XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gr

XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_

XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_

XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gr

...

a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
s

XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10

self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)


XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10

XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTest (10, 50)
self.theta_[i] (50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gradient=(50, 1)
XTrain=(10, 50)
YTrain=(10, 1)
a=(10, 1)
YHat=(10, 1)
loss=()
gr

In [60]:
class MetaSGD(object):
    def __init__(self):
        
        #initialize number of tasks i.e number of tasks we need in each batch of tasks
        self.num_tasks = 2
        
        #number of samples i.e number of shots  -number of data points (k) we need to have in each task
        self.num_samples = 10

        #number of epochs i.e training iterations
        self.epochs = 10000
        
        #hyperparameter for the inner loop (inner gradient update)
        self.alpha = 0.0001
        
        #hyperparameter for the outer loop (outer gradient update) i.e meta optimization
        self.beta = 0.0001
       
        #randomly initialize our model parameter theta
        self.theta = np.random.normal(size=85).reshape(85, 1)
         
        #randomly initialize alpha with same shape as theta
        self.alpha = np.random.normal(size=85).reshape(85, 1)
      
    #define our sigmoid activation function  
    def sigmoid(self,a):
        return 1.0 / (1 + np.exp(-a))
    
    
    #now let us get to the interesting part i.e training :P
    def train(self, XTrain, YTrain, XTest, YTest):
        
        #for the number of epochs,
        for e in range(self.epochs):        
            
            self.theta_ = []
            
            #for task i in batch of tasks
            for i in range(self.num_tasks):
               
                #sample k data points and prepare our train set
                
                a = np.matmul(XTrain, self.theta)

                YHat = self.sigmoid(a)

                #since we are performing classification, we use cross entropy loss as our loss function
                print("first", np.matmul(-YTrain.T, np.log(YHat)))
                print("second", np.matmul((1 -YTrain.T), np.log(1 - YHat)))
                print("third", self.num_samples)

                loss = ((np.matmul(-YTrain.T, np.log(YHat)) - np.matmul((1 -YTrain.T), np.log(1 - YHat)))/self.num_samples)
                print("loss", loss.values[0])
                loss = loss.values[0]
                #minimize the loss by calculating gradients
                gradient = np.matmul(XTrain.T, (YHat - YTrain)) / self.num_samples
                print(f"gradient={gradient.shape}")

                #update the gradients and find the optimal parameter theta' for each of tasks
                print("self.theta", self.theta)
                print("self.alpha", self.alpha, self.alpha.shape)
                print("np.multiply(self.alpha,gradient)", np.multiply(self.alpha,gradient))
                self.theta_.append(self.theta - (np.multiply(self.alpha,gradient)))
                
     
            #initialize meta gradients
            meta_gradient = np.zeros(self.theta.shape)
                        
            for i in range(self.num_tasks):
            
                #sample k data points and prepare our test set for meta training
                

                #predict the value of y
                print("XTest", XTest.shape)
                print("self.theta_[i]", self.theta_[i].shape)
                a = np.matmul(XTest, self.theta_[i])
                print("a", a)
                YPred = self.sigmoid(a)
                           
                #compute meta gradients
                meta_gradient += np.matmul(XTest.T, (YPred - YTest)) / self.num_samples

            #update our randomly initialized model parameter theta with the meta gradients
            self.theta = self.theta-self.beta*meta_gradient/self.num_tasks
                       
            #update our randomly initialized hyperparameter alpha with the meta gradients
            self.alpha = self.alpha-self.beta*meta_gradient/self.num_tasks
                                       
            if e%1000==0:
                print("Epoch {}: Loss {}\n".format(e,loss))
                print('Updated Model Parameter Theta\n')
                print('Sampling Next Batch of Tasks \n')
                print('---------------------------------\n')


In [61]:

df_preprocessed, features, target_feature = load_data()
df_preprocessed = df_preprocessed.dropna(subset=['target_binary_intrusion'], how='any')

X_train, X_test, y_train, y_test = train_test_split(df_preprocessed[features], df_preprocessed['target_binary_intrusion'],
                                      test_size=0.15, stratify=df_preprocessed['target_binary_intrusion'])
X_train, X_test = cv_preprocessing(X_train, X_test)

model = MetaSGD()
model.train(X_train, y_train, X_test, y_test)


first 0    380.658467
dtype: float64
second 0   -889.907901
dtype: float64
third 10
loss 127.05663676566542
gradient=(85, 459)
self.theta [[ 1.15710508]
 [-1.13212167]
 [ 0.7762565 ]
 [-0.62587772]
 [-0.24166616]
 [ 0.70410894]
 [ 1.01397524]
 [-0.40276279]
 [ 0.89363729]
 [-1.19952922]
 [-0.52175359]
 [-0.28524174]
 [ 0.97128746]
 [ 0.96845188]
 [-2.1996222 ]
 [ 1.33300043]
 [ 0.54159632]
 [-1.21439239]
 [-1.32414412]
 [-1.1537911 ]
 [ 0.896206  ]
 [ 0.46103489]
 [-1.01303763]
 [-1.00839931]
 [-0.52340734]
 [-0.29556856]
 [-2.46486537]
 [-0.61694716]
 [ 0.25467251]
 [ 0.58487358]
 [ 0.72967536]
 [-0.45726209]
 [-1.48335624]
 [-1.17184933]
 [-1.89368178]
 [-1.42807067]
 [ 0.49068715]
 [-1.01891875]
 [-0.20096315]
 [ 0.1722676 ]
 [-0.15137067]
 [ 0.06963443]
 [-1.03844761]
 [-0.17857584]
 [ 1.13413839]
 [-0.67675425]
 [ 2.24087889]
 [ 0.19113607]
 [-0.75482824]
 [ 0.96679084]
 [ 1.58544466]
 [-0.31555541]
 [-0.33648542]
 [-0.72184744]
 [ 0.60039993]
 [-0.70053622]
 [ 0.67353651]
 [ 0.20

<ipython-input-60-54563e598628>:48: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  print("first", np.matmul(-YTrain.T, np.log(YHat)))
<ipython-input-60-54563e598628>:49: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to kee

ValueError: non-broadcastable output operand with shape (85,1) doesn't match the broadcast shape (85,495)

In [62]:
self.alpha,gradient

NameError: name 'self' is not defined